In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [7]:
import getpass
import os

if not os.environ.get("Groq_API_Key"):
    os.environ["GROQ_API_KEY"]=getpass.getpass("Enter API key for Groq:")

In [8]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [18]:
from langchain.llms import 
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

ImportError: cannot import name 'llm' from 'langchain.llms' (c:\Users\divya\OneDrive\Project(MCQgen)\mcqgen\lib\site-packages\langchain\llms\__init__.py)

In [10]:
RESPONSE_JSON = {
    "1": {
        "Q1": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "2": {
        "Q2": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "3": {
        "Q3": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
}

In [11]:
template = """
Text:{text}
You are an expert AI MCQ maker. From the above given text, it is you job to \
create a quiz of {number} multiple choice questions for {subject} students in a {difficult} level.
Make Sure the questions do not get repeated and check all the questions to be vomforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

# can write this as well,
# template = """
# You are an AI teacher. Based on the following text, create 3 multiple-choice questions (MCQs) with the correct answers. 
# The questions should be relevant and based on the key concepts in the text. The format should be as follows:
# 1. Question: {question}
#    a) Option 1
#    b) Option 2
#    c) Option 3
#    d) Option 4
#    Correct answer: {correct_answer}

# Text: {text}

# Please generate the MCQs.
# """

In [12]:
Quiz_Generation_Template = PromptTemplate(
    input_variables=["text", "number", "subject", "level", "response_json"],
    template=template
)

In [17]:
Quiz_Chain=LLMChain(llm=llm, prompt=Quiz_Generation_Template, output_keys="Quiz", verbose=True)

NameError: name 'llm' is not defined

In [19]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [20]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=template
    )

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[Quiz_Chain, review_chain], input_variables=["text", "number", "subject", "level", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [23]:
file_path=r"C:\Users\divya\OneDrive\Project(MCQgen)\data.txt"

In [24]:
file_path

'C:\\Users\\divya\\OneDrive\\Project(MCQgen)\\data.txt'

In [25]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [26]:
print(TEXT)

The Tournament of Power was originally conceived by Zeno after witnessing the Tournament of Destroyers held by Beerus and Champa. After enjoying the tournament for its entertainment value and the spectacle of the participating fighters, Zeno determined that a larger competition involving all of the Universes would prove even more entertaining.

Later, after meeting the Future Zeno, Zeno had more or less forgotten about the tournament until reminded by an eager Goku. Having not seen the original martial arts competition, the Future Zeno was uncertain of the idea, leading to the need for the Zeno Expo, a smaller demonstration of a fighting competition.

It was at the Zeno Expo where the stakes of the Tournament of Power were set forth. Per the wishes of the two Zenos, the Grand Minister announced that the eight universes with the lowest fighter levels would be required to compete. The Universes that did not win the competition would immediately be erased from existence by the two Omni-Ki

In [27]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"Q1": "Multiple Choice Question", "Options": {"a": "choose this", "b": "choose this", "c": "choose this", "d": "choose this"}, "Correct": "Correct Answer"}, "2": {"Q2": "Multiple Choice Question", "Options": {"a": "choose this", "b": "choose this", "c": "choose this", "d": "choose this"}, "Correct": "Correct Answer"}, "3": {"Q3": "Multiple Choice Question", "Options": {"a": "choose this", "b": "choose this", "c": "choose this", "d": "choose this"}, "Correct": "Correct Answer"}}'

In [28]:
NUMBER=5 
SUBJECT="biology"
LEVEL="Medium"